# 016 - Custom Profiler for Analysis of code execution

## This code looks at making a tailored profiler to surgically measure performance on a given function, saving the results to a file for additional analysis.

In [24]:
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
# - Google Colab Check
# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
import datetime

RunningInCOLAB = False
RunningInCOLAB = "google.colab" in str(get_ipython())
current_time = datetime.datetime.now()

if RunningInCOLAB:
    print(
        f"You are running this notebook in Google Colab at {current_time} in the {BOLD_START}{PROJECT_ID}{BOLD_END}lab."
    )
else:
    print(
        f"You are likely running this notebook with Jupyter iPython runtime at {current_time} in the {PROJECT_ID} lab."
    )


NameError: name 'PROJECT_ID' is not defined

## Library Management
### Load Libraries necessary for this operation via pip install

In [ ]:
# Import key libraries necessary to support dynamic installation of additional libraries
import sys

# Use subprocess to support running operating system commands from the program, using the "bang" (!)
# symbology is supported, however that does not translate to an actual python script, this is a more
# agnostic approach.
import subprocess
import importlib.util

### Library Import

In [ ]:
# - Import additional libraries that add value to the project related to NLP

# - Set of libraries that perhaps should always be in Python source
import backoff
import datetime
from dotenv import load_dotenv
import gc
import getopt
import glob
import inspect
import io
import itertools
import json
import math
import os
from pathlib import Path
import pickle
import platform
import random
import re
import shutil
import string
from io import StringIO
import subprocess
import socket
import sys
import textwrap
import tqdm
import traceback
import warnings
import time
import uuid

#- Datastructures
from dataclasses import dataclass, field

#- Profiling
from time import perf_counter
import gc
import io
import tracemalloc
import psutil
import cProfile
import pstats
from pstats import SortKey

#- Text formatting
from rich import print as rprint
from rich.console import Console
from rich.traceback import install
from tabulate import tabulate
import locale

# - Displays system info
from watermark import watermark as the_watermark
from py3nvml import py3nvml

# - Additional libraries for this work
import math
from base64 import b64decode
from IPython.display import Image, Markdown
import pandas, IPython.display as display, io, jinja2, base64
from IPython.display import clear_output  # used to support real-time plotting
import requests
import unidecode
import pydot

# - Data Science Libraries
import pandas as pd
import numpy as np
import polars as pl
import dask as da
import dask.dataframe as dd
import dask.bag as db
import xarray as xr
import cupy_xarray  # never actually invoked in source itself use ds=ds.cupy.as_cupy()
import pystac as pys
import pystac
from pystac.utils import datetime_to_str

# from stacframes import df_from
import fastparquet as fq
import zarr
from zarr import Group
import netCDF4 as nc
from netCDF4 import Dataset

try:
    import cudf
except Exception as e:
    pass

try:
    import cupy
except Exception as e:
    pass

# Tensorflow and related AI libraries
import tensorflow as tf
from tensorflow import data as tf_data

# Torch
import torch

# - Graphics
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.cbook import get_sample_data
from matplotlib.offsetbox import AnnotationBbox, DrawingArea, OffsetImage, TextArea
from matplotlib.pyplot import imshow
from matplotlib.patches import Circle
from PIL import Image as PIL_Image
import PIL.ImageOps
import matplotlib.image as mpimg
from imageio import imread
import seaborn as sns

from mpl_toolkits.basemap import Basemap
from pylab import *

# - Image meta-data for Section 508 compliance
import piexif
from piexif.helper import UserComment

# - Progress bar
from tqdm import tqdm
from tqdm.notebook import trange, tqdm


## DataClasses

In [25]:
## Dataclass used to represent each metric used during execution
#
@dataclass
class runtime_metrics:
    id: str

    # see @Profile
    runtime: float = field(init=False, default=0.0)

    # reference: https://docs.python.org/3/library/profile.html
    profile_data: cProfile.Profile = field(init=False)

    # reference: https://www.geeksforgeeks.org/how-to-get-file-size-in-python/
    file_size: float = field(
        init=False,
        default=0.0,
    )

    # milliseconds, reference: https://stackoverflow.com/questions/24723092/using-python-to-measure-in-situ-read-write-speed-for-files
    io_disk_read_time: float = field(
        init=False,
        default=0.0,
    )

    # milliseconds, reference: https://stackoverflow.com/questions/24723092/using-python-to-measure-in-situ-read-write-speed-for-files
    io_disk_write_time: float = field(
        init=False,
        default=0.0,
    )
    
    # number read operations[end-begin], reference: https://stackoverflow.com/questions/24723092/using-python-to-measure-in-situ-read-write-speed-for-files
    io_disk_read_count: float = field(
        init=False,
        default=0.0,
    )

    # number read operations[end-begin], reference: https://stackoverflow.com/questions/24723092/using-python-to-measure-in-situ-read-write-speed-for-files
    io_disk_write_count: float = field(
        init=False,
        default=0.0,
    )

    # bytes read [end-begin], reference: https://stackoverflow.com/questions/24723092/using-python-to-measure-in-situ-read-write-speed-for-files
    io_disk_read_throughput: float = field(
        init=False,
        default=0.0,
    )

    # bytes read [end-begin], reference: https://stackoverflow.com/questions/24723092/using-python-to-measure-in-situ-read-write-speed-for-files
    io_disk_write_throughput: float = field(
        init=False,
        default=0.0,
    )

    # number read operations[end-begin], reference: https://stackoverflow.com/questions/24723092/using-python-to-measure-in-situ-read-write-speed-for-files
    io_os_read_count: float = field(
        init=False,
        default=0.0,
    )

    # number read operations[end-begin], reference: https://stackoverflow.com/questions/24723092/using-python-to-measure-in-situ-read-write-speed-for-files
    io_os_write_count: float = field(
        init=False,
        default=0.0,
    )

    # milliseconds, reference: https://stackoverflow.com/questions/24723092/using-python-to-measure-in-situ-read-write-speed-for-files
    io_os_read_time: float = field(
        init=False,
        default=0.0,
    )

    # milliseconds, reference: https://stackoverflow.com/questions/24723092/using-python-to-measure-in-situ-read-write-speed-for-files
    io_os_write_time: float = field(
        init=False,
        default=0.0,
    )

    # calculated in MBs, reference: https://www.geeksforgeeks.org/monitoring-memory-usage-of-a-running-python-program/
    mem_current: float = field(
        init=False,
        default=0.0,
    )

    # calculated in MBs, reference: https://www.geeksforgeeks.org/monitoring-memory-usage-of-a-running-python-program/
    mem_peak: float = field(
        init=False,
        default=0.0,
    )

    def __str__(self):
        return f"""
                Id---------------------------------------------
                                      Id: {self.id}
                Runtime----------------------------------------
                                 Runtime: {self.runtime:,.2f} milliseconds

                I/O Size---------------------------------------
                               File Size: {self.file_size:,.2f} bytes

                I/O Counts-------------------------------------
                      Targeted disk read: {self.io_disk_read_count:,.2f} counts
                     Targeted disk write: {self.io_disk_write_count:,.2f} counts
                       General disk read: {self.io_os_read_count:,.2f} counts
                      General disk write: {self.io_os_write_count:,.2f} counts

                I/O Time---------------------------------------
                 Targeted disk read time: {self.io_disk_read_time:,.2f} milliseconds
                Targeted disk write time: {self.io_disk_write_time:,.2f} milliseconds
                  General disk read time: {self.io_os_read_time:,.2f} milliseconds
                 General disk write time: {self.io_os_write_time:,.2f} milliseconds

                Memory------------------------------------------
                                 Current: {self.mem_current:,.2f} MB
                                    Peak: {self.mem_peak:,.2f} MB

                """

    #def __repr__(self):
    #    return f'{self.__class__.__name__}(name={self.name!r}, unit_price={self.unit_price!r}, quantity={self.quantity_on_hand!r})'

    # TODO - CGW
    # def __post_init__(self):
    #    self.id = f'{self.phrase}_{self.word_type.name.lower()}'

    # worthy consideration - https://www.geeksforgeeks.org/psutil-module-in-python/

### Profile

In [26]:
# Profiling function custom created to track IO, memory, and runtme.
# Reference: https://jiffyclub.github.io/snakeviz/
# Reference: https://www.machinelearningplus.com/python/cprofile-how-to-profile-your-python-code/
# Reference: https://cloud.google.com/stackdriver/docs/instrumentation/setup/python
# Reference: https://www.turing.com/kb/python-code-with-cprofile

def profile(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):

        # custom metrics values
        current_memories = 0.0
        peak_memories = 0.0
        current_metric = runtime_metrics(id=func.__name__)
        disk = "sdc"

        #####################################################################################################
        # - Cprofiler startup
        # Reference: https://www.google.com/search?client=firefox-b-1-d&q=python+example+use+of+cprofle+for+a+single+function#cobssid=s
        #####################################################################################################
        pr = cProfile.Profile()
        pr.enable()
        start_time = time.perf_counter()

        #####################################################################################################
        # - Memory tracking
        #  Reference: https://docs.python.org/3/library/tracemalloc.html
        #  Reference: https://www.kdnuggets.com/how-to-trace-memory-allocation-in-python
        #  Reference: https://www.geeksforgeeks.org/monitoring-memory-usage-of-a-running-python-program/
        #####################################################################################################
        tracemalloc.start()

        #####################################################################################################
        # - Disk tracking
        # Reference: https://stackoverflow.com/questions/16945664/insight-needed-into-python-psutil-output#:~:text=1%20Answer%201%20%C2%B7%20read_count:%20number%20of,write_bytes:%20number%20of%20bytes%20written%20%C2%B7%20read_time:
        #####################################################################################################
        iocnt1 = psutil.disk_io_counters(perdisk=True)[disk]
        disk_io_counters1 = psutil.disk_io_counters()
        read_bytes_start = iocnt1.read_bytes
        write_bytes_start = iocnt1.write_bytes
        read_counters_start = iocnt1.read_count
        write_counters_start = iocnt1.write_count
        read_time_start = iocnt1.read_time
        write_time_start = iocnt1.write_time
        
        read_os_bytes_start = disk_io_counters1.read_bytes
        write_os_bytes_start = disk_io_counters1.write_bytes
        read_os_counters_start = disk_io_counters1.read_count
        write_os_counters_start = disk_io_counters1.write_count
        read_os_time_start = disk_io_counters1.read_time
        write_os_time_start = disk_io_counters1.write_time
        
        #####################################################################################################
        # - Actual function call
        #####################################################################################################
        result = func(*args, **kwargs)

        # disk close out
        # targeted I/O
        iocnt2 = psutil.disk_io_counters(perdisk=True)["sdc"]
        disk_io_counters2 = psutil.disk_io_counters()

        #targeted disk
        read_bytes_end = iocnt2.read_bytes
        write_bytes_end = iocnt2.write_bytes
        read_counters_end = iocnt2.read_count
        write_counters_end = iocnt2.write_count
        read_time_end = iocnt2.read_time
        write_time_end = iocnt2.write_time
        #general OS
        read_os_bytes_end = disk_io_counters2.read_bytes
        write_os_bytes_end = disk_io_counters2.write_bytes
        read_os_counters_end = disk_io_counters2.read_count
        write_os_counters_end = disk_io_counters2.write_count
        read_os_time_end = disk_io_counters2.read_time
        write_os_time_end = disk_io_counters2.write_time

        #targeted disk
        read_throughput = (read_bytes_end - read_bytes_start) / (1024 * 1024)  # MB/s
        write_throughput = (write_bytes_end - write_bytes_start) / (1024 * 1024)  # MB/s
        read_counters = (read_counters_end - read_counters_start)
        write_counters = (read_counters_end - read_counters_start)
        read_time =  (read_time_end - read_time_start)
        write_time = (write_time_end - write_time_start)
        current_metric.io_disk_read_throughput = read_throughput
        current_metric.io_disk_write_throughput = write_throughput
        current_metric.io_disk_read_count = read_counters
        current_metric.io_disk_write_count = write_counters
        current_metric.io_disk_read_time = read_time
        current_metric.io_disk_write_time = write_time

        #general OS
        read_os_throughput = (read_os_bytes_end - read_os_bytes_start) / (1024 * 1024)  # MB/s
        write_os_throughput = (write_os_bytes_end - write_os_bytes_start) / (1024 * 1024)  # MB/s
        read_os_counters = (read_os_counters_end - read_os_counters_start)
        write_os_counters = (read_os_counters_end - read_os_counters_start)
        read_os_time =  (read_os_time_end - read_os_time_start)
        write_os_time = (write_os_time_end - write_os_time_start)
        current_metric.io_os_read_throughput = read_os_throughput
        current_metric.io_os_write_throughput = write_os_throughput
        current_metric.io_os_read_count = read_os_counters
        current_metric.io_os_write_count = write_os_counters
        current_metric.io_os_read_time = read_os_time
        current_metric.io_os_write_time = write_os_time



        # memory close
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()        
        current_metric.mem_current = current / (1024 * 1024)
        current_metric.mem_peak = peak / (1024 * 1024)
        tracemalloc.clear_traces()


        # CProfiler disabled
        pr.disable()

        #can't pickle this result
        #current_metric.profile_data=pr
        
        # s = io.StringIO()
        # sortby = SortKey.CUMULATIVE
        # ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
        # ps.print_stats()
        # print(s.getvalue())
        end_time = time.perf_counter()
        # other characteristics
        current_metric.runtime = end_time - start_time

        timestamp = datetime.datetime.now()
        #timestamp_str = timestamp.strftime("%Y%m%d%H%M%S%f") # Format as string
        timestamp_str = timestamp.strftime("%Y%m%d%H%M%S") # Format as string
        unique_id = uuid.uuid4()
        filename = f"{output_directory}/{timestamp_str}_{unique_id}_{current_metric.id}_profiler.pkl"
        #print(filename)
        #print("##########################################")
        #print(current_metric)

        with open(filename, "wb") as file:
            pickle.dump(current_metric, file)

        return result

    return wrapper

## Function Declaration

#### Lib Diagnostics

In [27]:
def lib_diagnostics() -> None:

    import pkg_resources

    package_name_length = 20
    package_version_length = 10

    # Show notebook details
    #%watermark?
    #%watermark --github_username christophergwood --email christopher.g.wood@gmail.com --date --time --iso8601 --updated --python --conda --hostname --machine --githash --gitrepo --gitbranch --iversions --gpu
    # Watermark
    print(
        the_watermark(
            author=f"{AUTHOR_NAME}",
            github_username=f"GITHUB_USERNAME",
            email=f"{AUTHOR_EMAIL}",
            iso8601=True,
            datename=True,
            current_time=True,
            python=True,
            updated=True,
            hostname=True,
            machine=True,
            gitrepo=True,
            gitbranch=True,
            githash=True,
        )
    )

    print(f"{BOLD_START}Packages:{BOLD_END}")
    print("")
    # Get installed packages
    the_packages = [
        "nltk",
        "numpy",
        "os",
        "pandas",
        "keras",
        "seaborn",
        "fastparquet",
        "zarr",
        "dask",
        "pystac",
        "polars",
        "xarray",
    ]  # Functions are like legos that do one thing, this function outputs library version history of effort.

    installed = {pkg.key: pkg.version for pkg in pkg_resources.working_set}
    for package_idx, package_name in enumerate(installed):
        if package_name in the_packages:
            installed_version = installed[package_name]
            print(
                f"{package_name:<40}#: {str(pkg_resources.parse_version(installed_version)):<20}"
            )

    try:
        print(f"{'TensorFlow version':<40}#: {str(tf.__version__):<20}")
        print(
            f"{'     gpu.count:':<40}#: {str(len(tf.config.experimental.list_physical_devices('GPU')))}"
        )
        print(
            f"{'     cpu.count:':<40}#: {str(len(tf.config.experimental.list_physical_devices('CPU')))}"
        )
    except Exception as e:
        pass

    try:
        print(f"{'Torch version':<40}#: {str(torch.__version__):<20}")
        if torch.cuda.is_available():
            device = torch.device("cuda")
            print(f"{'     GPUs available?':<40}#: {torch.cuda.is_available()}")
            print(f"{'     count':<40}#: {torch.cuda.device_count()}")
            print(f"{'     current':<40}#: {torch.cuda.get_device_name(0)}")
        else:
            device = torch.device("cpu")
            print("No GPU available, using CPU.")
    except Exception as e:
        pass

    try:
        print(f"{'OpenAI Azure Version':<40}#: {str(the_openai_version):<20}")
    except Exception as e:
        pass

    return

#### Section 508 Compliance Tools

In [28]:
# Routines designed to support adding ALT text to an image generated through Matplotlib.


def capture(figure):
    buffer = io.BytesIO()
    figure.savefig(buffer)
    # return F"data:image/png;base64,{base64.b64encode(buffer.getvalue()).decode()}"
    return f"data:image/jpg;base64,{base64.b64encode(buffer.getvalue()).decode()}"


def make_accessible(figure, template, **kwargs):
    return display.Markdown(
        f"""![]({capture(figure)} "{template.render(**globals(), **kwargs)}")"""
    )


# requires JPG's or TIFFs
def add_alt_text(image_path, alt_text):
    try:
        if os.path.isfile(image_path):
            img = PIL_Image.open(image_path)
            if "exif" in img.info:
                exif_dict = piexif.load(img.info["exif"])
            else:
                exif_dict = {}

            w, h = img.size
            if "0th" not in exif_dict:
                exif_dict["0th"] = {}
            exif_dict["0th"][piexif.ImageIFD.XResolution] = (w, 1)
            exif_dict["0th"][piexif.ImageIFD.YResolution] = (h, 1)

            software_version = " ".join(
                ["STEM-001 with Python v", str(sys.version).split(" ")[0]]
            )
            exif_dict["0th"][piexif.ImageIFD.Software] = software_version.encode(
                "utf-8"
            )

            if "Exif" not in exif_dict:
                exif_dict["Exif"] = {}
            exif_dict["Exif"][piexif.ExifIFD.UserComment] = UserComment.dump(
                alt_text, encoding="unicode"
            )

            exif_bytes = piexif.dump(exif_dict)
            img.save(image_path, "jpeg", exif=exif_bytes)
        else:
            rprint(
                f"Cound not fine {image_path} for ALT text modification, please check your paths."
            )

    except (FileExistsError, FileNotFoundError, Exception) as e:
        process_exception(e)


# Appears to solve a problem associated with GPU use on Colab, see: https://github.com/explosion/spaCy/issues/11909
def getpreferredencoding(do_setlocale=True):
    return "UTF-8"

#### Libary Configuration

In [29]:
def set_library_configuration() -> None:

    ############################################
    # - JUPYTER NOTEBOOK OUTPUT CONTROL / FORMATTING
    ############################################
    # pandas set floating point to 4 places to things don't run loose
    debug.msg_info("Setting Pandas and Numpy library options.")
    pd.set_option(
        "display.max_colwidth", 10
    )  # None if you want to view the full json blob in the printed dataframe, use this
    pd.options.display.float_format = "{:,.4f}".format
    np.set_printoptions(precision=4)

#### Custom Exception Display

In [30]:
# this function displays the stack trace on errors from a central location making adjustments to the display on an error easier to manage
# functions perform useful solutions for highly repetitive code
def process_exception(inc_exception: Exception) -> None:
    if DEBUG_STACKTRACE == 1:
        traceback.print_exc()
        console.print_exception(show_locals=True)
    else:
        rprint(repr(inc_exception))

#### Example Complex Function to Profile

In [31]:
@profile
def complex_function():
    # Define the size of the matrix
    matrix_size = 2048
    # Generate two random matrices
    matrix_a = np.random.rand(matrix_size, matrix_size)
    matrix_b = np.random.rand(matrix_size, matrix_size)
    result_matrix = np.matmul(matrix_a, matrix_b)
    np.savez(
        "./folderOnColab/data/local_test.npy",
        the_matrix=result_matrix,
    )

NameError: name 'functools' is not defined

## Process

In [32]:
## Main routine that executes all code, does return a data frame of data for further analysis if desired.
#
#  @param (None)
def process() -> None:

    rprint(f"Entering {__name__} {inspect.stack()[0][3]}")

    #create output directory
    #update constants to reflect output directory
    
    for idx in range(0, ITERATIONS):
        complex_function()

    #read results and plot analysis
    source_filenames_list=[]
    print("...marshaling data files:")
    target_directory = "./folderOnColab/"
    if os.path.isdir(target_directory):
        for file in os.listdir(target_directory):
            if os.path.isfile(f"{target_directory}{os.sep}{file}"):
                print(f"......processing {file} from {target_directory}")
                filename, file_extension = os.path.splitext(file)
                if file_extension.lower() in ".pkl":
                    source_filenames_list.append(os.path.join(target_directory, file))

    for file in source_filenames_list:
        with open(file, 'rb') as f:
            x = pickle.load(f)
        print(x) 
        
    rprint(f"Exiting {__name__} {inspect.stack()[0][3]}")

# Main Routine (call all other routines)

In [54]:
if __name__ == "__main__":

    # note that this design now deviates from previous methods.
    # Implementation will assume a single execution of a single PIID folder, scanning results and
    # appending metrics to a single ASCII file as the code proceeds thus ensuring multi-processor, *nix driven execution.

    start_t = perf_counter()
    print("BEGIN PROGRAM")

    ############################################
    # CONSTANTS
    ############################################

    # Semantic Versioning
    VERSION_NAME = "PROFILER"
    VERSION_MAJOR = 0
    VERSION_MINOR = 0
    VERSION_RELEASE = 1

    DATA_VERSION_RELEASE = "-".join(
        [
            str(VERSION_NAME),
            str(VERSION_MAJOR),
            str(VERSION_MINOR),
            str(VERSION_RELEASE),
        ]
    )

    WORKING_FOLDER = "./folderOnColab"
    input_directory = "./folderOnColab"
    output_directory = "./folderOnColab"

    # Notebook Author details
    AUTHOR_NAME = "Christopher G Wood"
    GITHUB_USERNAME = "christophergarthwood"
    AUTHOR_EMAIL = "christopher.g.wood@gmail.com"

    # Encoding
    ENCODING = "utf-8"
    os.environ["PYTHONIOENCODING"] = ENCODING

    BOLD_START = "\033[1m"
    BOLD_END = "\033[0;0m"
    TEXT_WIDTH = 77

    DEBUG_STACKTRACE = 0
    ITERATIONS = 20

    THE_DEVICE_NAME = "/job:localhost/replica:0/task:0/device:CPU:0"

    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=UserWarning)

    # GPU Setup (for multiple GPU devices)
    device = torch.cuda.current_device()

    # softare watermark
    lib_diagnostics()

    # hardware specs
    get_hardware_stats()

    # - Core workhorse routine
    process()

    # - Save the results
    # save_output()

    end_t = perf_counter()
    print("END PROGRAM")
    print(f"Elapsed time: {end_t - start_t}")

BEGIN PROGRAM
Author: Christopher G Wood

Github username: GITHUB_USERNAME

Email: christopher.g.wood@gmail.com

Last updated: 2025-03-19T16:17:38.066451-05:00

Python implementation: CPython
Python version       : 3.12.9
IPython version      : 8.30.0

Compiler    : GCC 13.3.0
OS          : Linux
Release     : 5.15.167.4-microsoft-standard-WSL2
Machine     : x86_64
Processor   : x86_64
CPU cores   : 16
Architecture: 64bit

Hostname: ThulsaDoom

Git hash: 75ef482324151469f00ef6a7eabb56e013a98195

Git repo: git@github.com:christophergarthwood/jbooks.git

Git branch: Updates

Packages:

dask                                    #: 2024.12.1           
fastparquet                             #: 2024.11.0           
keras                                   #: 3.9.0               
numpy                                   #: 1.26.4              
pandas                                  #: 2.2.3               
polars                                  #: 1.24.0              
pystac                   

Entering __main__ get_hardware_stats

List Devices #########################################


I0000 00:00:1742419058.140159    2213 service.cc:148] XLA service 0x557a3fc6ae80 initialized for platform Host (this does not guarantee that XLA will be used). Devices:


[
    name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1436886498804413271
xla_global_id: -1
,
    name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 3998773451041774701
physical_device_desc: "device: XLA_CPU device"
xla_global_id: -1
,
    name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 16804952475028772928
physical_device_desc: "device: XLA_GPU device"
xla_global_id: -1
,
    name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4253024256
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6156346436551526819
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419

]

...

Entering __main__ read_netcdfs

Entering __main__ gather_variables

...pulling data for and stacking it to the core data structure:
......stacking ./folderOnColab/data/20250308_gfs.t00z.atmf000.nc
......stacking ./folderOnColab/data/20250309_gfs.t00z.atmf000.nc
......stacking ./folderOnColab/data/20250310_gfs.t00z.atmf000.nc
......stacking ./folderOnColab/data/20250311_gfs.t00z.atmf000.nc
......stacking ./folderOnColab/data/20250312_gfs.t00z.atmf000.nc
......stacking ./folderOnColab/data/20250313_gfs.t00z.atmf000.nc
......stacking ./folderOnColab/data/20250314_gfs.t00z.atmf000.nc
......stacking ./folderOnColab/data/20250315_gfs.t00z.atmf000.nc
......stacking ./folderOnColab/data/20250316_gfs.t00z.atmf000.nc
......stacking ./folderOnColab/data/20250317_gfs.t00z.atmf000.nc
...lat        data type:         <class 'numpy.ndarray'>
.......shape:(47185920,)
....datatype:float64
...lon        data type:         <class 'numpy.ndarray'>
.......shape:(47185920,)
....datatype:float64
...Data type           (cld_amt             )<class 'numpy.ndarray'>
.......shap

Exiting __main__ gather_variables

UnboundLocalError: cannot access local variable 'the_payload' where it is not associated with a value